In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [55]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')

label2num = {"democrat": 0, "republican": 1}
num2label = {0: "democrat", 1: "republican"}

# convert the target label to numeric
cong_voting["class"] = cong_voting["class"].apply(lambda x: label2num[x])

In [56]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [57]:
class ClassificationModel(nn.Module):
    def __init__(self, nodes_per_layer, input_shape, num_classes):
        super(ClassificationModel, self).__init__()

        layers = []
        layers.append(nn.Linear(input_shape, nodes_per_layer[0]))
        layers.append(nn.ReLU())

        for i in range(1, len(nodes_per_layer)):
            layers.append(nn.Linear(nodes_per_layer[i - 1], nodes_per_layer[i]))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(nodes_per_layer[-1], num_classes))
        layers.append(nn.Softmax(dim=1))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.to(self.model[0].weight.dtype)
        return self.model(x)

In [58]:
def train_model(model, train_loader, num_epochs=5, lr=0.001, val_loader=True) :
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')
        if val_loader is not None:
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_outputs = model(val_inputs)
                    _, predicted = torch.max(val_outputs.data, 1)
                    total += val_labels.size(0)
                    correct += (predicted == val_labels).sum().item()

            accuracy = correct / total
            print(f'Validation Accuracy: {accuracy}')
            
def predict(model, input_data):
    model.eval()
    with torch.no_grad():
        outputs = model(input_data)
        _, predicted = torch.max(outputs.data, 1)
    return predicted


In [61]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=200)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
    
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
    
    return train_X, train_Y, test_X, test_Y




In [64]:
# run the model for cong_voting dataset
train_X, train_Y, test_X, test_Y = train_test_split(cong_voting, "class", return_torch=True)

nodes_per_layer = [5, 7, 5]
input_shape = train_X.shape[1]
num_classes = 2 # binary classification
num_samples = train_X.shape[0] 

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
train_model(model, train_loader, lr= 0.01, val_loader=train_loader)

Epoch 1/5, Loss: 0.48547348380088806
Validation Accuracy: 0.6264367816091954
Epoch 2/5, Loss: 0.6333977580070496
Validation Accuracy: 0.6379310344827587
Epoch 3/5, Loss: 0.7927806973457336
Validation Accuracy: 0.6494252873563219
Epoch 4/5, Loss: 0.8300963640213013
Validation Accuracy: 0.6781609195402298
Epoch 5/5, Loss: 0.4682011604309082
Validation Accuracy: 0.6954022988505747


In [65]:
# run the model for wine_quality dataset
train_X, train_Y, test_X, test_Y = train_test_split(wine_quality, "wine_type", return_torch=True)

nodes_per_layer = [5, 7, 5]
input_shape = train_X.shape[1]
num_classes = 2 # binary classification
num_samples = train_X.shape[0] 

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
train_model(model, train_loader, lr= 0.01, val_loader=train_loader)

Epoch 1/5, Loss: 0.45223233103752136
Validation Accuracy: 0.9340130819545979
Epoch 2/5, Loss: 0.4559144973754883
Validation Accuracy: 0.9374759522893421
Epoch 3/5, Loss: 0.45908334851264954
Validation Accuracy: 0.9407464409388226
Epoch 4/5, Loss: 0.3135358691215515
Validation Accuracy: 0.9397845325125048
Epoch 5/5, Loss: 0.4563581049442291
Validation Accuracy: 0.9367064255482878
